### further 2022-2023 highlights

#### Advanced modelling

* extended parallelism, including parallel broadcasting to hierarchical data
* fully distributional probabilistic forecasts and metrics, skpro
* composable time series classifiers, regressors, distances, time series aligners
* benchmarking frameworks for comparing estimator performance

#### Marketplace and deployment features

* estimator search, estimator tags
* scikit-base interface for multiple libraries
* blueprint serialization and sharing
* fitted estimator serialization and sharing
* mlflow deployment via custom flavour

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Advanced modelling

## parallelism for multivariate and hierarchical broadcasting

univariate forecasters broadcast across variables if given multivariate data

example:

In [ ]:
from sktime.datasets import load_longley
from sktime.forecasting.arima import ARIMA

_, y = load_longley()

y = y.drop(columns=["UNEMP", "ARMED", "POP"])

forecaster = ARIMA()
forecaster.fit(y, fh=[1, 2, 3])

# forecasters_ is a data frame with fitted ARIMA models
# entries are references to individual isntances of ARIMA, per variable
forecaster.forecasters_

by default, this is base python loop ... but we can use parallel backend!

In [ ]:
forecaster = ARIMA()

# let's use joblib loky backend, with 2 workers
# parallelization configs are accessed via the scikit-base config interface

# backends are set via the backend:parallel config
forecaster.set_config(**{"backend:parallel": "loky"})  # or "multiprocessing", or "dask" (requires dask)
# backend params are set via the backend:parallel:params config
forecaster.set_config(**{"backend:parallel:params": {"n_jobs": 2}})  # passed to joblib.Parallel
# for documentation of the config interface, see set_config/get_config docstrings

In [ ]:
# fit/predict methods are now parallelized
forecaster.fit(y, fh=[1, 2, 3])
forecaster.forecasters_
# of course this is more useful for larger data

same for hierarchical data!

hierarchical = multiple time series by hierarchical scope or index, e.g., product line/category

(typical: 1.000s of low-level hierarchical categories)

![](./img/hierarchy.png)

In [ ]:
from hierarchical_demo_utils import load_product_hierarchy
from sktime.forecasting.model_selection import temporal_train_test_split

y = load_product_hierarchy()

y_train, y_test = temporal_train_test_split(y, test_size=4)
y_train

sliced at a specific date:

In [ ]:
# Multiindex slicing can become important when using hierarchical data!
y.loc[(slice(None), slice(None), "2000-01")]

Like for variables, `sktime` broadcasts simple models to hierarchical data:

In [ ]:
from sktime.forecasting.ets import AutoETS

forecaster = AutoETS(auto=True)

forecaster.fit(y_train, fh=[1, 2, 3, 4])
y_pred = forecaster.predict()
y_pred

In [ ]:
# forecasters_ has fitted ETS models
forecaster.forecasters_

In [ ]:
# parallelization is enabled via the same config interface as for variables
# (same backend is used for both variables and instances or hierarchy levels)
forecaster = AutoETS(auto=True)

# backends are set via the backend:parallel config
forecaster.set_config(**{"backend:parallel": "loky"})  # or "multiprocessing", or "dask" (requires dask)
# backend params are set via the backend:parallel:params config
forecaster.set_config(**{"backend:parallel:params": {"n_jobs": 2}})  # passed to joblib.Parallel
# for documentation of the config interface, see set_config/get_config docstrings

In [ ]:
# this is faster now!
forecaster.fit(y_train, fh=[1, 2, 3, 4])
y_pred = forecaster.predict()  # both fit and predict are parallelized

also works for:

* performance metrics (e.g., multivariate and hierarchical)
* transformation and preprocessing

side note: the same backend parameters are used for:

* embarrassingly parallel "special" estimators such as grid search, random search
* benchmarking and evaluation frameworks, e.g., `evaluate` for forecast benchmarks

estimator or function params are called:

* `backend`, string selecting backend, e.g., `loky`, `multiprocessing` or `dask`
* `backend_params`, dict with params passed to backend, e.g., `joblib.Parallel`

In [ ]:
# example: parallelizing grid search
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.performance_metrics.forecasting import MeanSquaredError
from sktime.split import ExpandingWindowSplitter

forecaster = ExponentialSmoothing()

cv = ExpandingWindowSplitter(fh=[1,2,3,4,5,6], initial_window=12, step_length=1)
param_grid = {
    "sp": [4, 6, 12],
    "seasonal": ["add", "mul"],
    "trend": ["add", "mul"],
    "damped_trend": [True, False],
}

gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv,
    backend="loky",
    backend_params={"n_jobs": 2},
    verbose=1,
    scoring=MeanSquaredError(square_root=True),
)

## probabilistic forecasting, distribution outputs, skpro

## modular time series distances, classifiers, aligners

Rich component relationships between object types!

* many classifiers, regressors, clusterers use distances or kernels
* distances and kernels are often composite, e.g., sum-of-distance, independent distance
* TS distances are often based on scalar multivariate distances (e.g., Euclidean)
* TS distances are often based on alignment, TS aligners are an estimator type!
* aligners internally typically use scalar uni/multivariate distances

example:

* 1-nn using `sklearn` nearest neighbors
* with multivariate dynamic time warping distance, from `dtw-python` library 
* on multivariate `"mahalanobis"` distance from `scipy`
* in `sktime` compatible interface, constructed from custom components

so, conceptually:

* we build an sequence alignment algorithm (`dtw-python`) using `scipy` Mahalanobis dist
* we get the distance matrix computation from alignment algorithm
* we use that distance matrix in `sklearn` knn
* together this is a time series classifier!

In [ ]:
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.dists_kernels.scipy_dist import ScipyDist

# Mahalanobis distance on R^n
mahalanobis_dist = ScipyDist(metric="mahalanobis")  # uses scipy distances

# pairwise multivariate aligner from dtw-python with Mahalanobis distance
mw_aligner = AlignerDTWfromDist(mahalanobis_dist)  # uses dtw-python

# turning this into alignment distance on time series
dtw_dist = DistFromAligner(mw_aligner)  # interface mutation to distance

# and using this distance in a k-nn classifier
clf = KNeighborsTimeSeriesClassifier(distance=dtw_dist)  # uses sklearn knn

works seamlessly with `get_params`, `set_params` for tuning!

In [ ]:
clf.get_params()

all object types are first class citizens in sktime!

* `"transformer-panel"` - time series distances, kernels, pairwise transformers on panel data
* `"transformer-pairwise"` for all pairwise transformers on tabular data, e.g., scalar distance
* `"aligner"` for all time series aligners
* `"transformer"` for all transformers, these can be composed with all the above

In [ ]:
from sktime.registry import all_estimators

all_estimators("transformer-pairwise-panel", as_dataframe=True, return_tags=["pwtrafo_type"])

In [ ]:
from sktime.registry import all_estimators

all_estimators("aligner", as_dataframe=True)

## Benchmarking - comparing estimator performance

the `benchmarking` module allows you to set up experiments to:

* compare the performance of one or more algorithms
* over one or multiple datasets
* against one or multiple performance metrics
* for a benchmark configuration defined by temporal resampling scheme


`sktime`'s `benchmarking` module is designed to:

* provide a high-level specification language
* prevent mistakes by abstracting away "dangerous" implementation details
* allow reproducible sharing of experiment setups and results

Any `sktime` compatible object can be plugged in!

Use `sktime` extension templates to add custom objects to experiment!

In [ ]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanSquaredPercentageError

# set up benchmark
benchmark = ForecastingBenchmark()

# add competing estimators
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=12),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=12),
    estimator_id="NaiveForecaster-last-v1",
)

# define tasks, for forecasting:
# backtesting schema, cv splitter, scorer, data
cv_splitter = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=12,
)
scorers = [MeanSquaredPercentageError()]
dataset_loaders = [load_airline]

# add task
for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

# run the experiment, write to csv
results_df = benchmark.run("./forecasting_results.csv")
results_df.T

for forecasting, use `evaluate` utility for smaller runs

# Marketplace and deployment features

* estimator search, estimator tags
* scikit-base interface
* blueprint serialization and sharing
* fitted estimator serialization and sharing
* mlflow deployment via custom flavour

## listing estimators, estimator search, estimator tags

* all objects now "first class citizens" with a type - scikit-base objects
* use `all_estimators` for search subset

example: list all forecasters (`sktime` native scope)

In [ ]:
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True)

or, list all splitters:

In [ ]:
all_estimators("splitter", as_dataframe=True)

all classes, objects come with tags:

In [ ]:
# class tags
from sktime.forecasting.arima import ARIMA

ARIMA.get_class_tags()
# interesting for users:
# object_type tells us this is a forecaster
# capability tags, e.g., "capability:insample", "capability:pred_int"

In [ ]:
# object tags
from sktime.forecasting.naive import NaiveForecaster

NaiveForecaster().get_tags()
# same tags
# values may depend on the object parameters, e.g., "handles-missing-data"
# class tags are "most general" capabilities

produce table with class tags:

In [ ]:
from sktime.registry import all_estimators

# list all forecasters, in a table, with two added columns
# capability:insample - can produce in-sample forecasts?
# capability:pred_int - can produce prediction intervals?
all_estimators(
    "forecaster", as_dataframe=True, return_tags=["capability:insample", "capability:pred_int"]
)

filter for class tags:

In [ ]:
# list all forecasters that can produce probabilistic forecasts
all_estimators(
    "forecaster", as_dataframe=True, filter_tags={"capability:pred_int": True}
)
# of course you can do this with simple pandas filtering too,
# or anything else you want to do with pandas, but it avoids tedious wrangling

roadmap, contribute!

* easy way to specify variable scope across packages, 1st, 2nd, and 3rd party
* updating estimator overview frontend

## sharing model blueprints and fitted models

how to share these with your friends?

* model blueprint specs, e.g., equivalent of spec `Pipeline([("foo", Foo()), ("bar", Bar(42))])`
* fitted models, e.g., state of `my_pipe.fit(y)` after the `fit` - specific to data!

### sharing model blueprints

blueprint specs can be serialized using simple string print - this contains all information!

In [ ]:
# let's define an example pipeline
from sktime.forecasting.compose._pipeline import TransformedTargetForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.impute import Imputer

pipe = TransformedTargetForecaster(
    steps=[
        ("imputer", Imputer()),
        ("forecaster", NaiveForecaster()),
    ]
)

In [ ]:
# serialize the pipeline to a string
# this is useful for logging and sharing
# pipe_str can be saved to a file, database, or shared over the internet
pipe_str = str(pipe)
pipe_str

for pseudo-random determinism, set any `random_state` parameters in the estimators

to deserialize, use `registry.craft` in the same python environment

for python environment, e.g., use `pip freeze`

In [ ]:
from sktime.registry import craft

pipe_new = craft(pipe_str)
pipe_new

this is the same estimator blueprint as `pipe`!

To compare blueprint, simply use the `==` operator (this is a `scikit-base` feature)

In [ ]:
pipe_new == pipe

sharing process:

* origin shares `pipe_str = str(pipe)` or `str(my_estimator)` and `pip freeze > requirements.txt` output
* recipient installs env from `requirements.txt` and runs `craft(pipe_str)` in that env

For custom estimators, in addition, the custom module needs to be shared.

Highly complex estimators can consist of multiple definition blocks - this is also supported by `craft` as follows.

Instead of a string conversion, we can also serialize:

In [ ]:
# pipe_spec is a string representation of the pipeline
# it can be stored in a file or a database like this
# the "return" statement indicates which object we store
# temporary variables like pipe, cv can be defined
pipe_spec = """
pipe = TransformedTargetForecaster(steps=[
    ("imputer", Imputer()),
    ("forecaster", NaiveForecaster())])
cv = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=[1, 2, 3])

return ForecastingGridSearchCV(
    forecaster=pipe,
    param_grid=[{
        "forecaster": [NaiveForecaster(sp=12)],
        "forecaster__strategy": ["drift", "last", "mean"],
    },
    {
        "imputer__method": ["mean", "drift"],
        "forecaster": [ThetaForecaster(sp=12)],
    },
    {
        "imputer__method": ["mean", "median"],
        "forecaster": [ExponentialSmoothing(sp=12)],
        "forecaster__trend": ["add", "mul"],
    },
    ],
    cv=cv,
    n_jobs=-1)
"""

In [ ]:
craft(pipe_spec)

some estimators require soft dependencies to be installed at `craft`

query required dependencies can *before* construction via `deps`:

In [ ]:
from sktime.registry import deps

deps(pipe_spec)

(if `pip freeze` is not ehough)

`imports` can be used to print a full import block:

In [ ]:
from sktime.registry import imports

imports(pipe_spec)  # the result can be copied above the spec in to a jupyter cell

### Persisting fitted models

to persist a fitted model:

In [ ]:
from sktime.datasets import load_airline

y = load_airline()

In [ ]:
# example pipeline
from sktime.forecasting.compose._pipeline import TransformedTargetForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.impute import Imputer

pipe = TransformedTargetForecaster(
    steps=[
        ("imputer", Imputer()),
        ("forecaster", NaiveForecaster()),
    ]
)

pipe.fit(y, fh=[1, 2, 3])

to serialize fitted objects, use `save` - default is `pkl`, but may differ for deep learning

* no args produces in-memory object
* `str` or `Path` arg will serialize to file

In [ ]:
pipe_mem = pipe.save()
# pipe_mem is a pickle

to deserialize use the `load` method on the memory object or a `str`, `Path`:

In [ ]:
from sktime.base import load

pipe_new = load(pipe_mem)

the loaded object can be used for prediction now.

In [ ]:
pipe_new.predict()

### mlflow custom flavour

END HERE, ADD SUMMARY

In [ ]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import 

benchmark = ForecastingBenchmark()

benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=12),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=12),
    estimator_id="NaiveForecaster-last-v1",
)

cv_splitter = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=12,
)

scorers = [MeanSquaredPercentageError()]

dataset_loaders = [load_airline]

for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

results_df = benchmark.run("./forecasting_results.csv")
results_df.T

### Instantiate an instance of a benchmark class
In this example we are comparing forecasting estimators.

In [ ]:
benchmark = ForecastingBenchmark()

### Add competing estimators
We add different competing estimators to the benchmark instance. All added estimators will 
be automatically ran through each added benchmark tasks, and their results compiled.

In [ ]:
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=12),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=12),
    estimator_id="NaiveForecaster-last-v1",
)

### Add benchmarking tasks
These are the prediction/validation tasks over which every estimator will be tested and their results compiled.

The exact arguments for a benchmarking task depend on the whether the objective is forecasting, classification, etc.,
but generally they are similar. The following are the required arguments for defining a forecasting benchmark task.

#### Specify cross-validation split regime(s)
Define cross-validation split regimes, using standard `sktime` objects. 

In [ ]:
cv_splitter = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=12,
)

#### Specify performance metric(s)
Define performance metrics on which to compare estimators, using standard `sktime` objects.

In [ ]:
scorers = [MeanSquaredPercentageError()]

#### Specify dataset loaders
Define dataset loaders, which are callables (functions) which should return a dataset. Generally
this is a callable which returns a dataframe containing the entire dataset. One can use
the `sktime` defined datasets, or define their own. Something as simple as the following
example will suffice: 
```python
def my_dataset_loader():
    return pd.read_csv("path/to/data.csv")
```
The datasets will be loaded when running the benchmarking tasks, ran through the cross-validation
regime(s) and subsequently the estimators will be tested over the dataset splits.

In [ ]:
dataset_loaders = [load_airline]

#### Add tasks to the benchmark instance
Use the previously defined objects to add tasks to the benchmark instance.
Optionally use loops etc. to easily setup multiple benchmark tasks reusing arguments.

In [ ]:
for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

### Run all task-estimator combinations and store results

Note that `run` won't rerun tasks it already has results for, so adding a new
estimator and running `run` again will only run tasks for that new estimator.

In [ ]:
results_df = benchmark.run("./forecasting_results.csv")
results_df.T

### Credits:
